# Ch 17: Multiple Dispatch

In this chapter we will discuss the use of type declarations in Julia and introduce methods, which are ways to implement different behavior for a function depending on the types of its arguments. This is called multiple dispatch.

### Julia's Type System

Julia programs manipulate *values*.  Every value has two parts: a *type* part and a *data* part. The type part answers the question "what kind of thing is this?", and the data part distinguishes one thing of a certain kind from every other thing of that kind.

#### DataType

In [2]:
typeof(3)

Int64

In this case the type is `Int64` and the data part is the bits `...0011`.

In Julia types are also values:

In [3]:
typeof(Int64)

DataType

In [4]:
typeof(DataType)

DataType

In fact, the identity `typeof(typeof(x)) === DataType` holds for all values in Julia. `DataType` is the backbone of the entire system. It does many jobs, which can be identified by looking inside a `DataType` object:

#### DataType Job 1: A symbolic description

This consists of a name (which is mostly a string), and a vector of sub-components:

In [5]:
T_comp = typeof(1+2im)

Complex{Int64}

In [6]:
T_comp.name

Complex

In [7]:
T_comp.parameters

svec(Int64)

#### DataType Job 2: Describe the representation

In [8]:
T_comp.types

svec(Int64, Int64)

In [9]:
T_comp.name.names

svec(:re, :im)

In [10]:
T_comp.size

16

In [11]:
T_comp.mutable   # whether this was declared with `type` (vs. `immutable`)

false

In [13]:
T_comp.ninitialized

2

In [14]:
T_comp.layout

Ptr{Nothing} @0x000000000e207a30

Ptr{T}: A memory address referring to data of type `T`

In [15]:
typeof(T_comp.layout)

Ptr{Nothing}

In [16]:
?Nothing

search: Nothing nothing isnothing



```
Nothing
```

A type with no fields that is the type of [`nothing`](@ref).


#### DataType Job 3: A nominal hierarchy of types

DataTypes form a tree of declared type relationships:

In [7]:
T_comp.super

Number

In [8]:
T_comp.super.super  # `Any` is the built-in top of the hierarchy.

Any

Let's explore this hierarchy (super- vs sub-) of types a little further.  

A supertype and a subtype has 'IS-A' relationship, similar to the relatonship between the parent class and its children classes in Python and Java.

Every type has a super type:

In [19]:
typeof(5)

Int64

In [20]:
Int64.super

Signed

In [21]:
Signed.super

Integer

In [22]:
Integer.super

Real

In [23]:
Real.super

Number

In [24]:
Number.super

Any

In [17]:
subtypes(Number)

2-element Array{Any,1}:
 Complex
 Real   

### Abstract vs. Concrete Types:

Concrete types are those that can create instances.  Like `Int64` or `Float64`.

A type `T` is concrete if there could be some value `x` such that `typeof(x) === T`. This is also sometimes called a "leaf type".

We often need a way to group related concrete types. In Julia this is done by defining an abstract type that serves as a parent
or super type. This is called **subtyping**.

So, in quick summary, *Abstract* types can have declared subtypes, while *concrete* types can have instances.


In [25]:
Number.abstract, Real.abstract, Integer.abstract, Signed.abstract, Int64.abstract

(true, true, true, true, false)

All of these type, except for Int64, are abstract.

A programmer can also define an abstract type and its concrete subtypes:

In [26]:
abstract type PointLike end  # # just a name and position in the type hierarchy

struct PointConcrete <: PointLike  # <: is the subtyping operator
    x
    y
end

### Type Declarations

The :: operator attaches type annotations to expressions and variables:

In [1]:
(1 + 2) :: Float64

TypeError: TypeError: in typeassert, expected Float64, got Int64

In [2]:
(1 + 2) :: Int64

3

The :: operator can also be appended to the left-hand side of an assignment, or as part of a declaration.

In [3]:
function returnfloat()
    x::Float64 = 100
    x
end

returnfloat (generic function with 1 method)

In [4]:
x = returnfloat()

100.0

In [5]:
typeof(x)

Float64

A type annotation can also be attached to the header of a function definition:

In [6]:
function sinc(x)::Float64
    if x == 0
        return 1
    end
    sin(x)/(x)
end

sinc (generic function with 1 method)

The return value of sinc is always converted to type Float64.

The default behavior in Julia when types are omitted is to allow values to be of any type ( Any ).

### Methods

In Julia, all named functions are generic functions.  This means that they are built up from many small methods.

**A method is a function definition with a specific signature (number and types of arguments)**.

We previously defined a struct named MyTime and a function named `printtime`:

In [1]:
using Printf

struct MyTime
    hour :: Int64
    minute :: Int64
    second :: Int64
end

function printtime(time)
    @printf("%02d:%02d:%02d", time.hour, time.minute, time.second)
end

printtime (generic function with 1 method)

As you can see, type declarations can and should be added for performance reasons to the fields in a struct definition.

To call this function, you have to pass a MyTime object as an argument:

In [2]:
start = MyTime(9, 45, 0)

MyTime(9, 45, 0)

In [3]:
printtime(start)

09:45:00

If we do not, then we get an error:

In [4]:
printtime("9:45:00")

ErrorException: type String has no field hour

To add a method to the function `printtime` that only accepts as argument a MyTime object, all we have to do is append :: followed by MyTime to the argument time in the function definition:

In [5]:
function printtime(time::MyTime)
    @printf("%02d:%02d:%02d", time.hour, time.minute, time.second)
end

printtime (generic function with 2 methods)

`printtime` is a method with one argument of type MyTime. Calling the function printtime with a MyTime object yields the same result:

In [6]:
printtime(start)

09:45:00

We can now redefine the first method without the :: type annotation allowing an argument of any type:

In [7]:
function printtime(time)
    println("I don't know how to print the argument time.")
end

printtime (generic function with 2 methods)

If you call the function `printtime` with an object different from MyTime , you get now:

In [8]:
printtime(150)

I don't know how to print the argument time.


In [9]:
methods(printtime)

# 2 methods for generic function "printtime":
[1] printtime(time::MyTime) in Main at In[5]:2
[2] printtime(time) in Main at In[7]:2

### Exercise 17-1

Rewrite `timetoint` and `inttotime` to specify their argument.

In [13]:
function timetoint(time::MyTime)
    
    minutes = time.hour * 60 + time.minute
    seconds = minutes * 60 + time.second
    
end


function inttotime(seconds::Int64)
    
    (minutes, second) = divrem(seconds, 60)
    (hour, minute) = divrem(minutes, 60)
    MyTime(hour, minute, second)
    
end

inttotime (generic function with 1 method)

### Additional Examples

Here’s a version of increment rewritten to specify its arguments:

In [17]:
function increment(time::MyTime, seconds::Int64)
    seconds += timetoint(time)
    inttotime(seconds)
end

increment (generic function with 1 method)

Note that now, it is a pure function, not a modifier. Here’s how you would invoke increment:

In [12]:
start = MyTime(9, 45, 0)
increment(start, 1337)

MyTime(10, 7, 17)

If you put the arguments in the wrong order, you get an error:

In [13]:
increment(1337, start)

MethodError: MethodError: no method matching increment(::Int64, ::MyTime)

The signature of the method is increment(time::MyTime, seconds::Int64) and not increment(seconds::Int64,
time::MyTime).

We may use *multiple dispatch* to handle this:

In [18]:
function increment(seconds::Int64, time::MyTime)
    increment(time, seconds)
end

increment (generic function with 2 methods)

In [16]:
increment(1337, start)

MyTime(10, 7, 17)

Rewriting `isafter` to act only on MyTime objects is as easy:

In [14]:
function isafter(t1::MyTime, t2::MyTime)
    (t1.hour, t1.minute, t1.second) > (t2.hour, t2.minute, t2.second)
end

isafter (generic function with 1 method)

### Constructor

A constructor is a special function that is called to create an object. When we define a type (struct), Julia automatically generates default constructors.

The default constructor methods of MyTime have the following signatures:

```
MyTime(hour, minute, second)
MyTime(hour::Int64, minute::Int64, second::Int64)
```

We have already used them to create instances of MyTime.  Note that the default constructor's arguments are the fields of the struct in the order they are listed in the definition.

#### Outer constructor

We can also add our own outer constructor methods (defined outside the struct definition), which calls the default (or inner) constructor to construct an instance of a programmer-defined type:

In [19]:
function MyTime(time::MyTime)
    MyTime(time.hour, time.minute, time.second)
end

MyTime

In [20]:
methods(MyTime)

# 3 methods for generic function "(::Type)":
[1] MyTime(hour::Int64, minute::Int64, second::Int64) in Main at In[1]:4
[2] MyTime(time::MyTime) in Main at In[19]:2
[3] MyTime(hour, minute, second) in Main at In[1]:4

This method is called a copy constructor because the new MyTime object is a copy of its argument.

#### Inner constructor

To enforce invariants (conditions that should never change during the execution of a program), we need inner constructor methods:

In [8]:
struct MyTime
    
    hour :: Int64
    minute :: Int64
    second :: Int64

    function MyTime(hour::Int64=0, minute::Int64=0, second::Int64=0)
        @assert(0 ≤ minute < 60, "Minute is not between 0 and 60.")
        @assert(0 ≤ second < 60, "Second is not between 0 and 60.")
        new(hour, minute, second)
    end
    
end

In [22]:
methods(MyTime)

# 6 methods for generic function "(::Type)":
[1] MyTime() in Main at In[21]:8
[2] MyTime(hour::Int64) in Main at In[21]:8
[3] MyTime(hour::Int64, minute::Int64) in Main at In[21]:8
[4] MyTime(time::MyTime) in Main at In[19]:2
[5] MyTime(hour::Int64, minute::Int64, second::Int64) in Main at In[21]:8
[6] MyTime(hour, minute, second) in Main at In[1]:4

Note that the inner constructor has optional arguments with default values.  That is why we have 4 methods associated with this inner constructor ( (), (hour::Int64), (hour::Int64, minute::Int64), (hour::Int64, minute::Int64, second::Int64) )

An inner constructor method is always defined inside the block of a type declaration and it has access to a special function called `new` that creates objects of the newly declared type.

**WARNING**:  The default constructor is not available if any inner constructor is defined. You have to write explicitly all the inner constructors you need.

A second method without arguments of the local function `new` exists:

In [24]:
mutable struct MMyTime
    
    hour :: Int
    minute :: Int
    second :: Int
    
    function MMyTime(hour::Int64=0, minute::Int64=0, second::Int64=0)
        @assert(0 ≤ minute < 60, "Minute is between 0 and 60.")
        @assert(0 ≤ second < 60, "Second is between 0 and 60.")
        
        time = new()
        time.hour = hour
        time.minute = minute
        time.second = second
        time
    end
    
end

This allows to construct recursive data structures, i.e. a struct where one of the fields is the struct itself. In this case the struct has to be mutable because its fields are modified after instantiation.

### show

`show` is a special function that returns a string representation of an object. For example, here is a show method for MyTime objects:

In [15]:
using Printf
function Base.show(io::IO, time::MyTime)
    @printf(io, "%02d:%02d:%02d", time.hour, time.minute, time.second)
end

The prefix Base is needed because we want to add a new method to the Base.show function. 

When you print an object, Julia invokes the show function:

In [26]:
time = MyTime(9, 45)

09:45:00

**TIP**: When we write a new composite type, we  usually start by writing an outer constructor, which makes it easier to
instantiate objects, and `show`, which is useful for debugging.

### Exercise 17-2

Write an inner constructor method for the Point class that takes `x` and `y` as optional parameters and assigns them to the corresponding fields.

In [33]:
struct Point
    
    x
    y
    
    function Point(x = 0, y = 0)
        print("hello from inner  ")
        new(x,y)
    end
    
end

In [34]:
p1 = Point(1)

hello from inner  

Point(1, 0)

In [35]:
methods(Point)

# 3 methods for generic function "(::Type)":
[1] Point() in Main at In[33]:7
[2] Point(x) in Main at In[33]:7
[3] Point(x, y) in Main at In[33]:7

If we want to do this using an outer constructor

In [36]:
function OuterPoint(x = 1, y = 1)
    Point(x,y)
end

OuterPoint (generic function with 3 methods)

In [37]:
p = OuterPoint()

hello from inner  

Point(1, 1)

In [38]:
methods(OuterPoint)

# 3 methods for generic function "OuterPoint":
[1] OuterPoint() in Main at In[36]:2
[2] OuterPoint(x) in Main at In[36]:2
[3] OuterPoint(x, y) in Main at In[36]:2

**CAUTION:** Note that we name this outer constructor `OuterPoint` and NOT `Point`.  

***If we had named this `Point`, then it would create an infinite recursion.  This is because the outer constructor and the inner constructor have the same interface in this example.  The outer constructor would recursively call itself rather than the inner constructor.***

Well, let's go ahead and see this by defining an outer constructor with the same name and interface as the inner constructor.

In [39]:
#function Point(x = 1, y = 1)
    
    #print("hello from outer ")
#    Point(x,y)  
    # this was meant to call the inner (or default) constructor but it instead reursively calls the outer constructor.
    
#end

In [40]:
#p = Point()  # will give stack overflow error!!!

In [41]:
#methods(Point)

**Outer constructors therefore make sense when they have interfaces that do not overlap those of default (or inner) constructors.**

See Ch 17 - 2 AdvTypeSystem for further discussion.

In [42]:
function Point(x=-10)
    
    y = 5
    Point(x,y)
    
end

Point

In [43]:
methods(Point)

# 3 methods for generic function "(::Type)":
[1] Point() in Main at In[42]:3
[2] Point(x) in Main at In[42]:3
[3] Point(x, y) in Main at In[33]:7

In [45]:
Point()  # we define the outer constructor later.  It calls the inner constructor with (x=-10 and y=5).

hello from inner  

Point(-10, 5)

### Operator Overloading

By defining operator methods, you can specify the behavior of operators on programmer-defined types. 

For example, if you define a method named + with two MyTime arguments, you can use the + operator on MyTime objects. Here is what the definition might look like:

In [9]:
import Base.+
function +(t1::MyTime, t2::MyTime)
    seconds = timetoint(t1) + timetoint(t2)
    inttotime(seconds)
end

+ (generic function with 162 methods)

In [11]:
start = MyTime(9, 45);
duration = MyTime(1, 35, 0);


In [16]:
start + duration

11:20:00

When you apply the + operator to MyTime objects, Julia invokes the newly added method. When the result is shown, Julia invokes `show`. So there is a lot happening behind the scenes!

Adding to the behavior of an operator so that it works with programmer-defined types is called **operator overloading**.

### Multiple Dispatch

In the previous section we added two MyTime objects, but you also might want to add an integer to a MyTime object:

In [20]:
function +(time::MyTime, seconds::Int64)
    increment(time, seconds)
end

+ (generic function with 163 methods)

In [21]:
start = MyTime(9, 45)
start + 1337

10:07:17

Addition is a commutative operator so we have to add another method.

In [22]:
function +(seconds::Int64, time::MyTime)
    time + seconds
    # or we could do
    # increment(seconds, time) or increment(time,seconds)
end

+ (generic function with 164 methods)

In [23]:
1337 + start

10:07:17

The choice of which method to execute when a function is applied is called *dispatch*. 

Julia allows the dispatch process to choose which of a function’s methods to call based on the number of arguments given, and on the types of all of the function’s arguments. Using all of a function’s arguments to choose which method should be invoked is known as **multiple dispatch**.

### Exercise 17-3

Write + methods for point objects:

If both operands are point objects, the method should return a new point object whose `x` coordinate is the sum of the
`x` coordinates of the operands, and likewise for the `y` coordinates.

If the first or the second operand is a tuple, the method should add the first element of the tuple to the `x` coordinate
and the second element to the `y` coordinate, and return a new point object with the result.

In [27]:
function +(p1::Point, p2::Point)
    x = p1.x + p2.x
    y = p1.y + p2.y
    Point(x,y)
end


function +(t::Tuple, p::Point)
    x = t[1] + p.x
    y = t[2] + p.y
    Point(x,y)
end


function +(p::Point, t::Tuple)
#     x = t[1] + p.x
#     y = t[2] + p.y
#     Point(x,y)
    
    t + p # since Tuple + Point was defined above
    
end

    

+ (generic function with 167 methods)

In [28]:
p1 = Point(1,2)
p2 = Point(3,4)

p1+p2

Point(4, 6)

In [29]:
p1 + (5,6)

Point(6, 8)

In [30]:
(5,6) + p1

Point(6, 8)

### Generic Programming

Multiple dispatch is useful when it is necessary, but (fortunately) it is not always necessary. Often you can avoid it by writing functions that work correctly for arguments with different types. Many of the functions we wrote for strings also work for other sequence types. For example, in Dictionary as a Collection of Counters we used histogram to count the number of times each letter appears in a word.

In [32]:
function histogram(s)
    d = Dict()
    for c in s
        if c ∉ keys(d)
            d[c] = 1
        else
            d[c] += 1
        end
    end
    d
end

histogram (generic function with 1 method)

This function also works for lists, tuples, and even dictionaries, as long as the elements of `s` are hashable, so they can be
used as keys in `d`.

In [33]:
t = ("spam", "egg", "spam", "spam", "bacon", "spam")
histogram(t)

Dict{Any,Any} with 3 entries:
  "bacon" => 1
  "spam"  => 4
  "egg"   => 1

### Polymorphism

Functions that work with several types are called polymorphic.  Polymorphism can facilitate code reuse. 

We have already seen an example of polymorphic behavior when we overloaded `+` operator such that it works on instances of MyTime.

As another example, the built-in function `sum`, which adds the elements of a sequence, works as long as the elements of the sequence support addition.  Since a + method is provided for MyTime objects, they work with `sum`:

In [34]:
t1 = MyTime(1, 7, 2)
t2 = MyTime(1, 5, 8)
t3 = MyTime(1, 5, 0)
sum((t1, t2, t3))

03:17:10

In general, if all of the operations inside a function work with a given type, the function works with that type.

The best kind of polymorphism is the unintentional kind, where you discover that a function you already wrote can be applied to a type you never planned for.

### Interface and Implementation

One of the goals of multiple dispatch is to make software more maintainable, which means that you can keep the program working when other parts of the system change, and modify the program to meet new requirements. A design principle that helps achieve that goal is to keep interfaces separate from implementations. This means that the methods having an argument annotated with a type should not depend on how the fields of that type are represented.

After you deploy a new type, you might discover a better implementation. If other parts of the program are using your type, it might be time-consuming and error-prone to change the interface. 

But if you designed the interface carefully, you can change the implementation without changing the interface, which means that other parts of the program don’t have to change.


### More Examples

In [2]:
struct Tiger
    taillength::Float64
    coatcolor  # not including a type annotation is the same as `::Any`
end

tigger = Tiger(3.5, "orange")

Tiger(3.5, "orange")

In [3]:
abstract type Cat end
subtypes(Cat)

0-element Array{Any,1}

In [4]:
struct Lion <: Cat  # Lion is a subtype of Cat
    maneColor
    roar::AbstractString
end

In [5]:
subtypes(Cat)

1-element Array{Any,1}:
 Lion

In [6]:
Lion(roar::AbstractString) = Lion("green", roar)
# This is an outer constructor because it's outside the type definition.
# Note that this outer constructor has only one argument (as opposed to two in the default constructors)

Lion

In [7]:
struct Panther <: Cat  # Panther is also a subtype of Cat
    eyeColor
    Panther() = new("green")
    # Panthers will only have this inner constructor, and no default constructor!!
end



In [8]:
function meow(animal::Lion)
    animal.roar  # access type properties using dot notation
end

function meow(animal::Panther)
    "grrr"
end

function meow(animal::Tiger)
    "rawwwr"
end

meow (generic function with 3 methods)

In [9]:
meow(tigger), meow(Lion("brown", "ROAAR")), meow(Panther())

("rawwwr", "ROAAR", "grrr")

In [10]:
# Review the local type hierarchy
Tiger <: Cat, Lion <: Cat, Panther <: Cat

(false, true, true)

In [11]:
function pet_cat(cat::Cat)
    println("The cat says $(meow(cat))")
end

pet_cat (generic function with 1 method)

In [12]:
pet_cat(Lion("42"))

The cat says 42


In [13]:
pet_cat(tigger)  # well, we did not define Tiger as a sub-type of Cat (yes, this is an intentional error)

MethodError: MethodError: no method matching pet_cat(::Tiger)
Closest candidates are:
  pet_cat(!Matched::Cat) at In[11]:2

In [14]:
# Let's define a function with more arguments, so we can see the difference

function fight(t::Tiger, c::Cat)
    println("The $(t.coatcolor) tiger wins!")
end

fight (generic function with 1 method)

In [15]:
fight(tigger, Panther())
fight(tigger, Lion("ROAR"))

The orange tiger wins!
The orange tiger wins!


In [16]:
# Let's change the behavior when the Cat is specifically a Lion

In [17]:
fight(t::Tiger, l::Lion) = println("The $(l.maneColor)-maned lion wins!")

fight (generic function with 2 methods)

In [18]:
methods(fight)

# 2 methods for generic function "fight":
[1] fight(t::Tiger, l::Lion) in Main at In[17]:1
[2] fight(t::Tiger, c::Cat) in Main at In[14]:4

In [19]:
fight(tigger, Panther())

The orange tiger wins!


In [20]:
fight(tigger, Lion("ROAR"))

The green-maned lion wins!


In [25]:
fight(Panther(), tigger)

MethodError: MethodError: no method matching fight(::Panther, ::Tiger)

In [21]:
fight(l::Lion, c::Cat) = println("The victorious cat says $(meow(c))")

fight (generic function with 3 methods)

In [22]:
methods(fight)

# 3 methods for generic function "fight":
[1] fight(t::Tiger, l::Lion) in Main at In[17]:1
[2] fight(t::Tiger, c::Cat) in Main at In[14]:4
[3] fight(l::Lion, c::Cat) in Main at In[21]:1

In [23]:
fight(Lion("balooga!"), Panther())

The victorious cat says grrr


In [26]:
fight(Panther(), Lion("RAWR"))

MethodError: MethodError: no method matching fight(::Panther, ::Lion)
Closest candidates are:
  fight(!Matched::Tiger, ::Lion) at In[17]:1
  fight(!Matched::Tiger, ::Cat) at In[14]:4
  fight(!Matched::Lion, ::Cat) at In[21]:1

In [27]:
fight(c::Cat, l::Lion) = println("The cat beats the Lion")

fight (generic function with 4 methods)

In [28]:
fight(Panther(), Lion("RAWR"))

The cat beats the Lion


In [30]:
methods(fight)

# 4 methods for generic function "fight":
[1] fight(t::Tiger, l::Lion) in Main at In[17]:1
[2] fight(t::Tiger, c::Cat) in Main at In[14]:4
[3] fight(l::Lion, c::Cat) in Main at In[21]:1
[4] fight(c::Cat, l::Lion) in Main at In[27]:1

In [31]:
fight(Lion("RAR"), Lion("brown", "rarrr"))

MethodError: MethodError: fight(::Lion, ::Lion) is ambiguous. Candidates:
  fight(c::Cat, l::Lion) in Main at In[27]:1
  fight(l::Lion, c::Cat) in Main at In[21]:1
Possible fix, define
  fight(::Lion, ::Lion)

In [32]:
fight(l::Lion, l2::Lion) = println("The lions come to a tie") 

fight(Lion("RAR"), Lion("brown", "rarrr"))

The lions come to a tie
